In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

In [2]:

## load data
products = pd.read_csv('/home/ashwin/Downloads/hackerearth/h1/cluster/cproducts.csv')
tender = pd.read_csv('/home/ashwin/Downloads/hackerearth/h1/cluster/ctender.csv')

In [3]:


print('product file has {} rows and {} columns'.format(products.shape[0], products.shape[1]))
print('tender file has {} rows and {} columns'.format(tender.shape[0], tender.shape[1]))

product file has 11972 rows and 16 columns
tender file has 149752 rows and 13 columns


In [4]:
print(products.isnull().sum())
print('----------------')
print(tender.isnull().sum())

customerID                       0
DOB                            319
Gender                        1028
State                         1583
PinCode                        817
transactionDate                  0
store_code                       0
store_description                0
till_no                          0
transaction_number_by_till       0
promo_code                       0
promotion_description         8510
product_code                     0
product_description              0
sale_price_after_promo           0
discountUsed                     0
dtype: int64
----------------
customerID                        0
DOB                            4596
Gender                         6663
State                         11650
PinCode                        5564
transactionDate                   0
store_code                        0
store_description                 0
till_no                           0
tender_type                       0
transaction_number_by_till        0
payment_amount

In [8]:

products['promotion_description'].fillna(products['promotion_description'].mode()[0], inplace=True)
products['Gender'].fillna(products['Gender'].mode()[0], inplace=True)
products['State'].fillna('no_state', inplace=True)
products['PinCode'].fillna(-1, inplace=True)
products['DOB'].fillna("1", inplace=True)


In [9]:
for c in products.columns:
    lbl = LabelEncoder()
    if products[c].dtype == 'object' and c not in ['store_description','customerID','transactionDate']:
        products[c] = lbl.fit_transform(products[c])

In [10]:
stores = list(set(products['store_code']))

cluster_labels = []
cluster_store = []
cluster_data = []
cluster_customers = []
cluster_score = []

for x in stores:
    cld = products[products['store_code'] == x]
    cluster_customers.append(cld['customerID'])
    cld.drop(['store_code','customerID','transactionDate','store_description'], axis=1, inplace=True)
    
    rbs = RobustScaler()
    cld2 = rbs.fit_transform(cld)
    
    km1 = KMeans(n_clusters=4)
    km2 = km1.fit(cld2)
    label = km2.predict(cld2)
    
    s_score = silhouette_score(cld2, label)
    cluster_score.append(s_score)
    
    cluster_labels.append(label)
    cluster_store.append(np.repeat(x, cld.shape[0]))
    cluster_data.append(cld2)

/home/ashwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [11]:
np.mean(cluster_score)

0.93365956065872291

In [12]:

## merge list into ndarray
cluster_data = np.concatenate(cluster_data)

In [13]:
cluster_data.shape

(11972, 12)

In [14]:
cluster_customers = np.concatenate(cluster_customers)
cluster_store = np.concatenate(cluster_store)
cluster_labels = np.concatenate(cluster_labels)

In [15]:

## create submission files
sub1 = pd.DataFrame({'customerID':cluster_customers, 'store_code':cluster_store, 'cluster':cluster_labels})

In [16]:
np.savetxt('/home/ashwin/Downloads/hackerearth/h1/cluster/subOne_18.txt', cluster_data)
sub1.to_csv('/home/ashwin/Downloads/hackerearth/h1/cluster/subtwo_18.csv', index=False)